# Script 2 to clean the C-CLAMP metadata file (after processing in OpenRefine)
This script has to be available within the same folder as `author_metadata_OpenRefine.tsv`

In [2]:
# Import the necessary packages
import pandas as pd
import numpy as np
import re
from itertools import chain

In [3]:
# Read the metadata file
metadata_df = pd.read_csv("author_metadata_OpenRefine.tsv", sep="\t", encoding='utf-8')
print(metadata_df)

                  Author              DOB          DOB_WIP         POB  \
0           Jan Engelman      7 juni 1900      7 juni 1900     Utrecht   
1            Willem Maas    28 april 1897    28 april 1897     Utrecht   
2           Joep Nicolas   6 oktober 1897   6 oktober 1897         NaN   
3          Albert Helman  7 november 1903  7 november 1903  Paramaribo   
4     Willem Nieuwenhuis             1886             1886         NaN   
...                  ...              ...              ...         ...   
8748        A. De Geyter       20ste eeuw       20ste eeuw         NaN   
8749            J. Hoing       20ste eeuw       20ste eeuw         NaN   
8750       Flor Kielbaey       20ste eeuw       20ste eeuw         NaN   
8751  Hendrik Imberechts    13 april 1922    13 april 1922         NaN   
8752            H. Aerts       20ste eeuw       20ste eeuw         NaN   

         POB_WIP              DOD          DOD_WIP        POD    POD_WIP  \
0        Utrecht    20 maart 1972  

## Step 1: Clean all dates in DOB_WIP and DOD_WIP

In [5]:
## Turn all centuries into years
# Create a function that takes centuries as input and turns them into approximated years
# The 18th century (18de eeuw), e.g., becomes 17xx
def century_to_year(century_string):
    match = re.search(r'(\d{1,2})(ste|de) eeuw', century_string)
    if match:
        century = int(match.group(1))

        # Centuries up to 10 are treated differently, a zero is prepended
        if 1 <= century <= 10:
            year_string = f"0{century - 1}xx"
        else:
            year = (century - 1) * 100
            year_string = f"{year // 100}xx"
        return century_string.replace(match.group(0), year_string)
    else:
        return century_string

# Change DOB_WIP values into strings
metadata_df['DOB_WIP'] = metadata_df['DOB_WIP'].astype(str)

# Apply the century_to_year function
metadata_df['DOB_WIP'] = metadata_df['DOB_WIP'].apply(century_to_year)

In [6]:
## Turn spelled out months into numbers
# Create a function that takes months in text form as input and turns them into numbers
# September, e.g., becomes 09
month_mapping = {
    "januari": "01", "februari": "02", "maart": "03", "april": "04", "mei": "05", "juni": "06",
    "juli": "07", "augustus": "08", "september": "09", "oktober": "10", "november": "11", "december": "12"
}

# For each month, find the corresponding number
def month_to_number(month):
    month = month.lower().strip()
    return month_mapping.get(month, month)

# Replace the months by their correct number
def replace_month(text):
    for month in month_mapping:
        if month.lower() in text.lower():
            text = re.sub(r'\b' + month + r'\b', month_to_number(month), text, flags=re.IGNORECASE)
    return text

# Apply the replace_month function to DOB_WIP
metadata_df['DOB_WIP'] = metadata_df['DOB_WIP'].apply(replace_month)

In [7]:
## Add a zero before days lower than 10
# Create a function to do so
def add_zero_day(date):
    return re.sub(r'\b(\d)\b', r'0\1', date)

# Apply the add_zero_day function to DOB_WIP
metadata_df['DOB_WIP'] = metadata_df['DOB_WIP'].apply(add_zero_day)

In [8]:
## Work towards the international notation: YYYY-MM-DD (approach for DOB_WIP and DOD_WIP)
# Keep missing values empty
metadata_df['DOB_WIP'] = metadata_df['DOB_WIP'].replace('nan', '')

# Define a function that creates the international notation
def international_date_A(date):
    if not date.strip():
        return ""

    if date.startswith("ca."):
        date = date[3:].strip()
        prefix = "ca. "
    elif date.startswith("voor"):
        date = date[4:].strip()
        prefix = "voor "
    elif date.startswith("na"):
        date = date[2:].strip()
        prefix = "na "
    else:
        prefix = ""

    match = re.match(r'((\d{2}\/?){1,2})\s((\d{2}\/?){1,2})\s(((-?\w{2,4}){,2}\/?){1,2})', date)
    if match:
        day = match.group(1)
        month = match.group(3)
        year = match.group(5)

        if year.startswith('-'):
            year = '-' + year[1:].zfill(4)
        else:
            year = year.zfill(4)

        return f"{prefix}{year}-{month}-{day}"

    match = re.match(r'((\d{2}\/?){1,2})\s(((-?\w{2,4}){,2}\/?){1,2})', date)
    if match:
        month = match.group(1)
        year = match.group(3)

        if year.startswith('-'):
            year = '-' + year[1:].zfill(4)
        else:
            year = year.zfill(4)

        return f"{prefix}{year}-{month}-xx"

    match = re.match(r'(((-?\w{2,4}){,2}\/?){1,2})', date)
    if match:
        year = match.group(1)

        if year.startswith('-'):
            year = '-' + year[1:].zfill(4)
        else:
            year = year.zfill(4)

        return f"{prefix}{year}-xx-xx"
    else:
        return date

# Apply the international_date_A function to DOB_WIP
metadata_df['DOB_WIP'] = metadata_df['DOB_WIP'].astype(str)
metadata_df['DOB_WIP'] = metadata_df['DOB_WIP'].apply(international_date_A)

In [9]:
# Apply all changes to DOD_WIP
metadata_df['DOD_WIP'] = metadata_df['DOD_WIP'].astype(str)
metadata_df['DOD_WIP'] = metadata_df['DOD_WIP'].apply(century_to_year)
metadata_df['DOD_WIP'] = metadata_df['DOD_WIP'].apply(replace_month)
metadata_df['DOD_WIP'] = metadata_df['DOD_WIP'].apply(add_zero_day)
metadata_df['DOD_WIP'] = metadata_df['DOD_WIP'].replace('nan', '')
metadata_df['DOD_WIP'] = metadata_df['DOD_WIP'].apply(international_date_A)

In [10]:
# Save the metadata file with cleaned DOB_WIP and DOD_WIP
metadata_df.replace(['', 'NA', 'nan'], np.nan, inplace=True)
metadata_df.to_csv("author_dates_list_A.txt", index=False, sep='\t', encoding='utf-8')

## Step 2: Clean all dates in birthDate_WIP and deathDate_WIP

In [12]:
# Reread the metadata file
metadata_df = pd.read_csv("author_dates_list_A.txt", sep="\t", encoding='utf-8')

In [13]:
# Apply the earlier defined add_zero_day function to birthDate_WIP
metadata_df['birthDate_WIP'] = metadata_df['birthDate_WIP'].astype(str)
metadata_df['birthDate_WIP'] = metadata_df['birthDate_WIP'].apply(add_zero_day)

In [14]:
## Reformat DD/MM/YYYY into YYYY-MM-DD (approach for birthDate_WIP and deathDate_WIP)
# Define a function
def international_date_B(date):
    match = re.match(r'(\d{2})/(\d{2})/(\d{4})', date)
    if match:
        month, day, year = match.groups()
        return f"{year}-{month}-{day}"
    else:
        return date

# Apply the international_date_B function to birthDate_WIP
metadata_df['birthDate_WIP'] = metadata_df['birthDate_WIP'].apply(international_date_B)

In [15]:
## Omit values containing 'ca' or a literal '/', which have been misprocessed
# Define a function
def omit_unwanted(date):
    if 'ca' in date or '/' in date or '?' in date:
        return ''
    else:
        return date

# Apply the omit_unwanted function to birthDate_WIP
metadata_df['birthDate_WIP'] = metadata_df['birthDate_WIP'].replace('nan', '')
metadata_df['birthDate_WIP'] = metadata_df['birthDate_WIP'].apply(omit_unwanted)

In [16]:
## Add unknown months and days to values that only contain years
# Define a function
def add_unknown_month_day(date):
    match = re.match(r'^(-?\w{2,4})$', date)
    if match:
        year = match.group(1)
        return f"{year}-xx-xx"
    else:
        return date

# Apply the add_unknown_month_day function to birthDate_WIP
metadata_df['birthDate_WIP'] = metadata_df['birthDate_WIP'].apply(add_unknown_month_day)

In [17]:
## Ensure that all dates are formatted as (-)YYYY-MM-DD, i.e., add zeros where needed
# Define a function
def correct_format(date):
    match = re.match(r'^(-?\w{1,4})-(\w{2})-(\w{2})$', date)
    if match:
        year, month, day = match.groups()

        if year.startswith('-'):
            year = '-' + year[1:].zfill(4)
        else:
            year = year.zfill(4)

        return f"{year}-{month}-{day}"
    else:
        return date

# Apply the correct_format function to birthDate_WIP
metadata_df['birthDate_WIP'] = metadata_df['birthDate_WIP'].apply(correct_format)

In [18]:
# Apply all changes to deathDate_WIP
metadata_df['deathDate_WIP'] = metadata_df['deathDate_WIP'].astype(str)
metadata_df['deathDate_WIP'] = metadata_df['deathDate_WIP'].apply(add_zero_day)
metadata_df['deathDate_WIP'] = metadata_df['deathDate_WIP'].apply(international_date_B)
metadata_df['deathDate_WIP'] = metadata_df['deathDate_WIP'].replace('nan', '')
metadata_df['deathDate_WIP'] = metadata_df['deathDate_WIP'].apply(omit_unwanted)
metadata_df['deathDate_WIP'] = metadata_df['deathDate_WIP'].apply(add_unknown_month_day)
metadata_df['deathDate_WIP'] = metadata_df['deathDate_WIP'].apply(correct_format)

In [19]:
# Save the metadata file with cleaned DOB_WIP and DOD_WIP, AND with cleaned birthDate_WIP and deathDate_WIP
metadata_df.replace(['', 'NA', 'nan'], np.nan, inplace=True)
metadata_df.to_csv("author_dates_list_B.txt", index=False, sep='\t', encoding='utf-8')

## Step 3: Clean all dates in geboortedatum_WIP and sterfdatum_WIP

In [21]:
# Reread the metadata file
metadata_df = pd.read_csv("author_dates_list_B.txt", sep="\t", encoding='utf-8')

In [22]:
## The values in geboortedatum_WIP and sterfdatum_WIP are formatted in the same way as DOB_WIP and DOD_WIP were,
## for which we can simply apply the previously defined functions
# geboortedatum_WIP
metadata_df['geboortedatum_WIP'] = metadata_df['geboortedatum_WIP'].astype(str)
metadata_df['geboortedatum_WIP'] = metadata_df['geboortedatum_WIP'].apply(century_to_year)
metadata_df['geboortedatum_WIP'] = metadata_df['geboortedatum_WIP'].apply(replace_month)
metadata_df['geboortedatum_WIP'] = metadata_df['geboortedatum_WIP'].apply(add_zero_day)
metadata_df['geboortedatum_WIP'] = metadata_df['geboortedatum_WIP'].replace('nan', '')
metadata_df['geboortedatum_WIP'] = metadata_df['geboortedatum_WIP'].apply(international_date_A)

# sterfdatum_WIP
metadata_df['sterfdatum_WIP'] = metadata_df['sterfdatum_WIP'].astype(str)
metadata_df['sterfdatum_WIP'] = metadata_df['sterfdatum_WIP'].apply(century_to_year)
metadata_df['sterfdatum_WIP'] = metadata_df['sterfdatum_WIP'].apply(replace_month)
metadata_df['sterfdatum_WIP'] = metadata_df['sterfdatum_WIP'].apply(add_zero_day)
metadata_df['sterfdatum_WIP'] = metadata_df['sterfdatum_WIP'].replace('nan', '')
metadata_df['sterfdatum_WIP'] = metadata_df['sterfdatum_WIP'].apply(international_date_A)

In [23]:
# Save the metadata file with all cleaned date columns
metadata_df.replace(['', 'NA', 'nan'], np.nan, inplace=True)
metadata_df.to_csv("author_dates_list_C.txt", index=False, sep='\t', encoding='utf-8')

## Step 4: Compare the values in the different date columns and select the best fit

In [25]:
# Reread the metadata file
metadata_df = pd.read_csv("author_dates_list_C.txt", sep="\t", encoding='utf-8')
metadata_df = metadata_df.fillna('')

In [26]:
## First, compare DOB_WIP with birthDate_WIP, and DOD_WIP with deathDate_WIP. Then, compare with geboortedatum_WIP and sterfdatum_WIP
# Define a function that compares two columns with dates
# It defaults to the value in column A, unless that in B is more specific
def compare_dates(row, col_A, col_B):
    value_A = row[col_A]
    value_B = row[col_B]

    if not value_B:
        return value_A

    x_count_A = value_A.count('x')
    x_count_B = value_B.count('x')

    if x_count_A <= x_count_B:
        return value_A
    else:
        return value_B

def apply_date_comparison(df, col_A, col_B, new_col):
    df[new_col] = df.apply(compare_dates, axis=1, col_A=col_A, col_B=col_B)
    return df

# Apply the first comparison
metadata_df['DOB_WIP'] = metadata_df['DOB_WIP'].astype(str)
metadata_df['DOD_WIP'] = metadata_df['DOD_WIP'].astype(str)
metadata_df['birthDate_WIP'] = metadata_df['birthDate_WIP'].astype(str)
metadata_df['deathDate_WIP'] = metadata_df['deathDate_WIP'].astype(str)

metadata_df = apply_date_comparison(metadata_df, 'DOB_WIP', 'birthDate_WIP', 'DOB_clean_A')
metadata_df = apply_date_comparison(metadata_df, 'DOD_WIP', 'deathDate_WIP', 'DOD_clean_A')

# Apply the second comparison
metadata_df['geboortedatum_WIP'] = metadata_df['geboortedatum_WIP'].astype(str)
metadata_df['sterfdatum_WIP'] = metadata_df['sterfdatum_WIP'].astype(str)

metadata_df = apply_date_comparison(metadata_df, 'DOB_clean_A', 'geboortedatum_WIP', 'DOB_clean_B')
metadata_df = apply_date_comparison(metadata_df, 'DOD_clean_A', 'sterfdatum_WIP', 'DOD_clean_B')

## Step 5: Compare the values in the different place columns to fill in the gaps

In [28]:
## First, compare POB_WIP with birthPlace_WIP, and POD_WIP with deathPlace_WIP. Then, compare with geb_plaats_WIP and overl_plaats_WIP
# Define a function to compare two columns
def compare_two(row, col_A, col_B):
    value_A = row[col_A]
    value_B = row[col_B]

    if not value_A:
        return value_B
    else:
        return value_A

def apply_two_comparison(df, col_A, col_B, new_col):
    df[new_col] = df.apply(compare_two, axis=1, col_A=col_A, col_B=col_B)
    return df

# Apply the first comparison
metadata_df = apply_two_comparison(metadata_df, 'POB_WIP', 'birthPlace_WIP', 'POB_clean_A')
metadata_df = apply_two_comparison(metadata_df, 'POD_WIP', 'deathPlace_WIP', 'POD_clean_A')

# Apply the second comparison
metadata_df = apply_two_comparison(metadata_df, 'POB_clean_A', 'geb_plaats_WIP', 'POB_clean_B')
metadata_df = apply_two_comparison(metadata_df, 'POD_clean_A', 'overl_plaats_WIP', 'POD_clean_B')

## Step 6: Compare the values in the different occupation columns to fill in the gaps

In [30]:
# We can apply the function 'apply_two_comparison' defined above
metadata_df = apply_two_comparison(metadata_df, 'occupation', 'profession', 'occupation_clean')

In [31]:
# Save the cleaned metadata file
metadata_df.replace(['', 'NA', 'nan'], np.nan, inplace=True)
metadata_df.to_csv("author_clean_full.txt", index=False, sep='\t', encoding='utf-8')

## Step 7: Omit any redundant columns

In [33]:
# Reread the metadata file
metadata_df = pd.read_csv("author_clean_full.txt", sep="\t", encoding='utf-8')

In [34]:
# Print all column labels
metadata_df.columns

Index(['Author', 'DOB', 'DOB_WIP', 'POB', 'POB_WIP', 'DOD', 'DOD_WIP', 'POD',
       'POD_WIP', 'Link', 'identifier', 'givenName', 'familyName',
       'alternateNames', 'occupation', 'birthPlace', 'birthPlace_WIP',
       'birthDate', 'birthDate_WIP', 'deathPlace', 'deathPlace_WIP',
       'deathDate', 'deathDate_WIP', 'gender', 'geb_plaats', 'geb_plaats_WIP',
       'birthCountry', 'birthCountry_WIP', 'overl_plaats', 'overl_plaats_WIP',
       'deathCountry', 'deathCountry_WIP', 'period', 'geboortedatum',
       'geboortedatum_WIP', 'sterfdatum', 'sterfdatum_WIP', 'language',
       'category', 'profession', 'organisation', 'DOB_clean_A', 'DOD_clean_A',
       'DOB_clean_B', 'DOD_clean_B', 'POB_clean_A', 'POD_clean_A',
       'POB_clean_B', 'POD_clean_B', 'occupation_clean'],
      dtype='object')

In [35]:
## Filter the DataFrame and rename the columns
# Filter
filtered_metadata_df = metadata_df[['Author', 'Link', 'identifier', 'givenName', 'familyName', 'alternateNames',
                                 'DOB_clean_B', 'POB_clean_B', 'birthCountry_WIP',
                                 'DOD_clean_B', 'POD_clean_B', 'deathCountry_WIP',
                                 'gender', 'occupation_clean', 'period', 'language', 'category', 'organisation']]

# Rename columns
filtered_metadata_df = filtered_metadata_df.rename(columns = {'DOB_clean_B': 'birthDate', 'POB_clean_B': 'birthPlace', 'birthCountry_WIP': 'birthCountry',
                                                       'DOD_clean_B': 'deathDate', 'POD_clean_B': 'deathPlace', 'deathCountry_WIP': 'deathCountry',
                                                       'occupation_clean': 'occupation'})

# Fill empty cells with NaN
filtered_metadata_df.replace(['', 'NA', 'nan'], np.nan, inplace=True)
filtered_metadata_df

,Author,Link,identifier,givenName,familyName,alternateNames,birthDate,birthPlace,birthCountry,deathDate,deathPlace,deathCountry,gender,occupation,period,language,category,organisation
0,Jan Engelman,https://www.dbnl.org/auteurs/auteur.php?id=eng...,enge016,Johannes Aloysius Antonius,Engelman,Jan Engelman,1900-06-07,Utrecht,NaN,1972-03-20,Amsterdam,NaN,male,redacteur/journalist/vertaler,Twintigste Eeuw,nl,secundair/nonfictie,NaN
1,Willem Maas,https://www.dbnl.org/auteurs/auteur.php?id=maa...,maas024,Willem Arnoldus,Maas,Willem Maas,1897-04-28,Utrecht,NaN,1950-05-06,Utrecht,NaN,male,illustrator/tekenaar/beeldend kunstenaar,Twintigste Eeuw,NaN,NaN,NaN
2,Joep Nicolas,https://www.dbnl.org/auteurs/auteur.php?id=nic...,nico008,Josephus Antonius Hubertus Franciscus,Nicolas,Joep Nicolas,1897-10-06,Roermond,NaN,1972-07-25,Steyl,NaN,male,schilder/ambachtsman,Twintigste Eeuw,NaN,nonfictie,NaN
3,Albert Helman,https://www.dbnl.org/auteurs/auteur.php?id=hel...,helm003,Albert,Helman,Albert Helman/Beckmesser/Brandaris/Floris Kapt...,1903-11-07,Paramaribo,Suriname,1996-07-10,Amsterdam,NaN,male,musicus/journalist/politicus/redacteur/staatsm...,Twintigste Eeuw,nl,jeugdliteratuur/nonfictie/suriname/buitenland,NaN
4,Willem Nieuwenhuis,https://www.dbnl.org/auteurs/auteur.php?id=nie...,nieu047,Willem,Nieuwenhuis,Willem Nieuwenhuis,1886-xx-xx,NaN,NaN,1935-xx-xx,NaN,NaN,male,NaN,Twintigste Eeuw,nl,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8748,A. De Geyter,https://www.dbnl.org/auteurs/auteur.php?id=gey...,geyt003,A.,De Geyter,A. De Geyter,19xx-xx-xx,NaN,NaN,NaN,NaN,NaN,male,NaN,Twintigste Eeuw,NaN,secundair,NaN
8749,J. Hoing,https://www.dbnl.org/auteurs/auteur.php?id=hoi...,hoin002,J.,Hoing,J. Hoing,19xx-xx-xx,NaN,NaN,NaN,NaN,NaN,male,NaN,Twintigste Eeuw,NaN,secundair/nonfictie,NaN
8750,Flor Kielbaey,https://www.dbnl.org/auteurs/auteur.php?id=kie...,kiel012,Flor,Kielbaey,Flor Kielbaey,19xx-xx-xx,NaN,NaN,NaN,NaN,NaN,male,vertaler,Twintigste Eeuw,NaN,secundair,NaN
8751,Hendrik Imberechts,https://www.dbnl.org/auteurs/auteur.php?id=imb...,imbe003,Hendrik,Imberechts,Hendrik Imberechts,1922-04-13,Muizen,NaN,2012-02-03,Leuven,NaN,male,rk-geestelijke/kloosterling,Twintigste Eeuw,nl,secundair/nonfictie,NaN


In [36]:
# Save the cleaned and filtered metadata file
filtered_metadata_df.to_csv("author_filtered_clean.txt", index=False, sep='\t', encoding='utf-8')